In [3]:
import os
import shutil
import random
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds # NEW: Import TFDS
import sys

! pip install -q tensorflow-datasets

IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 20
FINE_TUNE_EPOCHS = 40
NUM_CLASSES = 10
MODEL_NAME = 'satellite_resnet_FINETUNED_90acc.h5'
SEED = 42

In [5]:
ds_full = tfds.load('eurosat/rgb', split='train', shuffle_files=True, as_supervised=True, with_info=False)
ds_size = sum(1 for _ in ds_full)

def preprocess(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_full.take(int(0.8 * ds_size)).map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = ds_full.skip(int(0.8 * ds_size)).map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

conv_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

model = Sequential([
    conv_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

conv_base.trainable = False
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_base = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint_base = ModelCheckpoint(MODEL_NAME, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)


model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_val,
    callbacks=[early_stopping_base, checkpoint_base],
    verbose=1
)

conv_base.trainable = True

for layer in model.layers[0].layers:
    if layer.name.startswith('conv5') or layer.name.startswith('res5'):
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping_ft = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint_ft = ModelCheckpoint(MODEL_NAME, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)


model.fit(
    ds_train,
    epochs=EPOCHS + FINE_TUNE_EPOCHS,
    initial_epoch=EPOCHS,
    validation_data=ds_val,
    callbacks=[early_stopping_ft, checkpoint_ft],
    verbose=1
)

model.save(MODEL_NAME)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2242 - loss: 2.1284
Epoch 1: val_accuracy improved from -inf to 0.43389, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 35s 33ms/step - accuracy: 0.2245 - loss: 2.1274 - val_accuracy: 0.4339 - val_loss: 1.6791
Epoch 2/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3553 - loss: 1.7419
Epoch 2: val_accuracy did not improve from 0.43389
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.3553 - loss: 1.7418 - val_accuracy: 0.4331 - val_loss: 1.5692
Epoch 3/20
671/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3922 - loss: 1.6456
Epoch 3: val_accuracy improved from 0.43389 to 0.46056, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.3922 - loss: 1.6456 - val_accuracy: 0.4606 - val_loss: 1.5011
Epoch 4/20
671/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4017 - loss: 1.6198
Epoch 4: val_accuracy did not improve from 0.46056
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4018 - loss: 1.6198 - val_accuracy: 0.4485 - val_loss: 1.4660
Epoch 5/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4115 - loss: 1.5819
Epoch 5: val_accuracy improved from 0.46056 to 0.47796, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.4114 - loss: 1.5819 - val_accuracy: 0.4780 - val_loss: 1.4151
Epoch 6/20
674/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4162 - loss: 1.5656
Epoch 6: val_accuracy improved from 0.47796 to 0.50704, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.4162 - loss: 1.5656 - val_accuracy: 0.5070 - val_loss: 1.4030
Epoch 7/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4267 - loss: 1.5497
Epoch 7: val_accuracy did not improve from 0.50704
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4267 - loss: 1.5497 - val_accuracy: 0.4700 - val_loss: 1.3764
Epoch 8/20
673/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4266 - loss: 1.5377
Epoch 8: val_accuracy improved from 0.50704 to 0.51222, saving model to satellite_resnet_FINETUNED_90acc.h5


...

675/675 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.4468 - loss: 1.4994 - val_accuracy: 0.5437 - val_loss: 1.3289
Epoch 17/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4477 - loss: 1.4985
Epoch 17: val_accuracy did not improve from 0.54370
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4477 - loss: 1.4985 - val_accuracy: 0.5383 - val_loss: 1.3230
Epoch 18/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4538 - loss: 1.4816
Epoch 18: val_accuracy did not improve from 0.54370
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4538 - loss: 1.4816 - val_accuracy: 0.5280 - val_loss: 1.3021
Epoch 19/20
674/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4584 - loss: 1.4687
Epoch 19: val_accuracy did not improve from 0.54370
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4584 - loss: 1.4687 - val_accuracy: 0.5346 - val_loss: 1.3250
Epoch 20/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4477 - loss: 1.4882
Epoch 20: val_ac

675/675 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.4477 - loss: 1.4882 - val_accuracy: 0.5563 - val_loss: 1.2942
Epoch 21/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3177 - loss: 4.2561
Epoch 21: val_accuracy improved from -inf to 0.53185, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 48s 40ms/step - accuracy: 0.3178 - loss: 4.2536 - val_accuracy: 0.5319 - val_loss: 1.3916
Epoch 22/60
674/675 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4782 - loss: 1.4773
Epoch 22: val_accuracy improved from 0.53185 to 0.55519, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.4783 - loss: 1.4771 - val_accuracy: 0.5552 - val_loss: 1.2488
Epoch 23/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5388 - loss: 1.2820
Epoch 23: val_accuracy did not improve from 0.55519
675/675 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.5389 - loss: 1.2819 - val_accuracy: 0.5378 - val_loss: 1.2875
Epoch 24/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5790 - loss: 1.1610
Epoch 24: val_accuracy improved from 0.55519 to 0.60204, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.5790 - loss: 1.1610 - val_accuracy: 0.6020 - val_loss: 1.1143
Epoch 25/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6142 - loss: 1.0646
Epoch 25: val_accuracy did not improve from 0.60204
675/675 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.6142 - loss: 1.0646 - val_accuracy: 0.4319 - val_loss: 1.8208
Epoch 26/60
673/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6335 - loss: 1.0226
Epoch 26: val_accuracy did not improve from 0.60204
675/675 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.6335 - loss: 1.0225 - val_accuracy: 0.4894 - val_loss: 1.4973
Epoch 27/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6521 - loss: 0.9613
Epoch 27: val_accuracy did not improve from 0.60204
675/675 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.6521 - loss: 0.9613 - val_accuracy: 0.4235 - val_loss: 2.0925
Epoch 28/60
674/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6794 - loss: 0.9079
Epoch 28: val_ac